In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

In [2]:
# Cargar el DataFrame generado previamente
# Asegúrate de que el DataFrame tenga columnas ['x', 'y', 'z'] y ['interpolated_valor1', ..., 'interpolated_valor200']
df = pd.read_csv('interpolated_results.csv')

In [3]:
# Separar las características (X) y las etiquetas (y)
X = df[['x', 'y', 'z']].values  # Coordenadas (entradas)
y = df.iloc[:, 3:].values       # Energías (salidas)

In [4]:
# Dividir los datos en entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Escalar las características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
from sklearn.linear_model import Ridge  # Cambia esto a Lasso si prefieres Lasso

In [7]:
# Entrenar el modelo de regresión con regularización Ridge (puedes cambiarlo a Lasso si prefieres)
model = Ridge(alpha=1.0)  # 'alpha' controla la regularización
model.fit(X_train_scaled, y_train)

Ridge()

In [8]:
# Realizar predicciones
y_pred = model.predict(X_test_scaled)

In [9]:
# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Modelo de regresión lineal con Ridge:")
print(f" - MSE: {mse:.4f}")
print(f" - R²: {r2:.4f}")
print("-------------------------------")

Modelo de regresión lineal con Ridge:
 - MSE: 0.0002
 - R²: 0.7396
-------------------------------


In [10]:
# Si deseas guardar el modelo entrenado
import pickle
with open('modelo_regresion_lineal.pkl', 'wb') as f:
    pickle.dump(model, f)

In [11]:
# También puedes guardar las predicciones
predictions_df = pd.DataFrame(y_pred, columns=[f'pred_valor{i+1}' for i in range(y_pred.shape[1])])
predictions_df.to_csv('predicciones.csv', index=False)


In [12]:
import numpy as np

# Definir los límites del cubo con los valores mínimo y máximo de las coordenadas x, y, z
x_min, x_max = df['x'].min(), df['x'].max()
y_min, y_max = df['y'].min(), df['y'].max()
z_min, z_max = df['z'].min(), df['z'].max()

# Generar coordenadas aleatorias dentro de los límites del cubo
num_samples = 5  # Número de coordenadas aleatorias que quieres generar
random_coordinates = np.array([
    [np.random.uniform(x_min, x_max), 
     np.random.uniform(y_min, y_max), 
     np.random.uniform(z_min, z_max)]
    for _ in range(num_samples)
])

In [13]:
print(random_coordinates)

[[-0.01543134  0.17841915  0.16282822]
 [-0.56747863  0.89363683 -0.816469  ]
 [-0.40873264 -0.87988967 -0.69260007]
 [ 0.73245511 -0.98257541  0.04148408]
 [-0.67772451 -0.42943227 -0.15057472]]


In [14]:
# Predicción de la energía para las coordenadas aleatorias
predicted_energies = model.predict(random_coordinates)

print(predicted_energies)

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.00767451  0.07425907  0.12012996  0.09813538  0.10476224
   0.08869885  0.16677701  0.19188171  0.15100754  0.16275442  0.16300028
   0.10410996  0.09757724  0.09797345  0.08894098  0.08764693  0.10287282
   0.09844758  0.07092398  0.10768501  0.09524589  0.06961038  0.07228463
   0.06819465  0.05686416  0.0597225   0.07544725  0.07068068  0.06742815
   0.06762027  0.066287    0.06363553  0.05181267  0.05294552  0.05421016
   0.05025839  0.04364995  0.04507292  0.04369685  0.04234666  0.0400848
   0.03836391  0.04139835  0.03863319  0.0381152   0.03620605  0.03465019
   0.03276682  0.03069986  0.03348834  0.03014027  0.02858971  0.02870602
   0.02934039  0.02752012  0.02510191  0.02617481  0.02333508  0.02258048
   0.02175885  0.02095125  0.02291984  0.01896059  0.01865096  0.01949833
   0.01705827  0.01646579  0.01701807  0.01637696  0.01